In [ ]:
import torch


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from nltk.tokenize.treebank import TreebankWordDetokenizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
!pip install emoji
import emoji

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 51kB 8.2MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=ef92222256341b63242893c1d41c712cbbf7a366853dff805c8b8942eeda6cec
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [ ]:
#mounting drive for the OUTPUT directory

from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/'My Drive'


In [ ]:
#!pip install pytorch-transformers
import torch
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from textblob import TextBlob
from sklearn.model_selection import train_test_split
import re

In [ ]:
df = pd.read_excel('/content/gdrive/My Drive/545 project/20thApril/Copy of Final_Emojis.xlsx')
#df = pd.read_csv('/content/gdrive/My Drive/545 project/emotions1.csv')

In [ ]:
df.loc[df['Affect Dimension'] == 'anger','newlabel'] = 0

df.loc[df['Affect Dimension'] == 'joy','newlabel'] = 1

df.loc[df['Affect Dimension'] == 'sadness/fear','newlabel'] = 2

df.loc[df['Affect Dimension'] == 'neutral','newlabel'] = 3
print (df['newlabel'][6034])

In [ ]:
def is_emoji(s):
    count = 0
    for emoji1 in emoji.UNICODE_EMOJI:
        count += s.count(emoji1)
        if count > 1:
            return False
    return bool(count)

def token_emoji_conversion(text):
    words2 = []
    words = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    words11=words.copy()
    idx=0
    i2=0
    n=0
    for i1, word in enumerate(words11):
      #print(word)
      if i2!=0 and n!=0:
        words.insert(idx, word1)
        idx+=1
        n=0
      if i2==0 and i1!=0:
        idx+=1
      i2=0
      m=0
      n=0
      for c in word:
      #c for c in str if c in emoji.UNICODE_EMOJI
        if is_emoji(c) == True:
          i2+=1
          if m==0:
            try:
              del(words[idx])
            except:
              print(words, idx)
          if n!=0:
            words.insert(idx, word1)
            idx+=1
            n=0
          word1 = emoji.demojize(c)
          word1 = word1.replace('_', ' ')
          word1 = word1.replace(':', '')
          word1 = nltk.word_tokenize((word1))
          for i in range(len(word1)):
            words.insert(idx, word1[i])
            #print(i)
            idx+=1
          m+=1
        else:
          if n==0:
            word1 = c
          else:
            word1=word1+c
          n+=1
    for token in words:
        if re.search('[a-zA-Z]', token):
            words2.append(token)
    
    #TreebankWordDetokenizer().detokenize(output)
    return TreebankWordDetokenizer().detokenize(words2)

In [ ]:
df['Tweet']=df['Tweet'].apply(token_emoji_conversion)


In [ ]:
df.head()

In [ ]:
sentences  = []
for sentence in df['Tweet']:
  sentence = sentence+"[SEP] [CLS]"
  sentences.append(sentence)

In [ ]:
sentences[0]

In [ ]:
!pip install transformers

from transformers import XLNetTokenizer, XLNetForSequenceClassification, XLNetConfig
from sklearn.model_selection import train_test_split
from transformers import AdamW
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

In [ ]:
tokenizer  = XLNetTokenizer.from_pretrained('xlnet-base-cased',do_lower_case=False)
tokenized_text = [tokenizer.tokenize(sent) for sent in sentences]


In [ ]:
tokenized_text[0]

In [ ]:
tokenizer.tokenize("I am GONE MAD")

In [ ]:
ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]


In [ ]:
print(ids[0])
labels = df['newlabel'].values
print(labels[0])

In [ ]:
max1 = len(ids[0])
for i in ids:
  if(len(i)>max1):
    max1=len(i)
print(max1)
MAX_LEN = max1

In [ ]:
input_ids2 = pad_sequences(ids,maxlen=MAX_LEN,dtype="long",truncating="post",padding="post")

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(input_ids2,labels,test_size=0.20, random_state=42)

In [ ]:
print(len(input_ids2[0]))
print (input_ids2[0])

In [ ]:
Xtrain = torch.tensor(xtrain)
Ytrain = torch.tensor(ytrain, dtype=torch.long)
Xtest = torch.tensor(xtest)
Ytest = torch.tensor(ytest, dtype=torch.long)

In [ ]:
batch_size = 3

In [ ]:
train_data = TensorDataset(Xtrain,Ytrain)
test_data = TensorDataset(Xtest,Ytest)
loader = DataLoader(train_data,batch_size=batch_size)
test_loader = DataLoader(test_data,batch_size=batch_size)

** If the model has already been trained and checkpoint generated then uncomment the following 2 lines to load the model and skip model definition and training **

In [ ]:
#model.load_state_dict(torch.load('/content/gdrive/My Drive/545 project/20thApril/checkpoint_xlnet_emoji/latest.ckpt'))
#model.to(device)

**MODEL**

In [ ]:
config = XLNetConfig.from_pretrained('xlnet-base-cased')

model = XLNetForSequenceClassification(config)

model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=4)
model.cuda()

In [ ]:
device = torch.device("cuda")
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=4)


In [ ]:
optimizer = AdamW(model.parameters(),lr=2e-5)# We pass model parameters

In [ ]:
import torch.nn as nn
#criterion = nn.CrossEntropyLoss()

In [ ]:
import numpy as np
def flat_accuracy(preds,labels): 
    correct=0
    for i in range(0,len(labels)):
        if(preds[i]==labels[i]):
            correct += 1
    return (correct/len(labels))*100

In [ ]:
no_train = 0
epochs = 3
for epoch in range(epochs):
    model.train()
    loss1 = []
    steps = 0
    train_loss = []
    l = []
    for inputs,labels1 in loader :
        inputs.to(device)
        labels1.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss = criterion(outputs[0],labels1.to(device)).to(device)
        logits = outputs[1]
        #ll=outp(loss)
        [train_loss.append(p.item()) for p in torch.argmax(outputs[0],axis=1).flatten() ]#our predicted 
        [l.append(z.item()) for z in labels1]# real labels
        loss.backward()
        optimizer.step()
        loss1.append(loss.item())
        no_train += inputs.size(0)
        steps += 1
    print("Current Loss is : {} Step is : {} number of Example : {} Accuracy : {}".format(loss.item(),epoch,no_train,flat_accuracy(train_loss,l)))

Save the trained model.

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/My Drive/545 project/20thApril/checkpoint_xlnet_emoji/latest.ckpt')

Test() run the model on test_data and check accuracy

In [ ]:
acc = []
lab = []
logits = []
t = 0
for inp,lab1 in test_loader:
  inp.to(device)
  lab1.to(device)
  t+=lab1.size(0)
  outp1 = model(inp.to(device))
  logits.append(outp1[0][1])
  [acc.append(p1.item()) for p1 in torch.argmax(outp1[0],axis=1).flatten() ]
  [lab.append(z1.item()) for z1 in lab1]
print("Total Examples : {} Accuracy {}".format(t,flat_accuracy(acc,lab)))